# Mapping

In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

In [2]:
spacex_df = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

## TASK 1: Mark all launch sites on a map

In [3]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [4]:
# Nasa Coordinate
nasa_coordinate = [29.559684888503615, -95.0830971930759]
map_coordinate = [30.63635, -98.79073]
site_map = folium.Map(location=map_coordinate, zoom_start=5)
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#3240a8;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

# Launch Sites
for index, record in spacex_df.iterrows():
    
    site_map.add_child(
        folium.Circle( 
            [record.Lat, record.Long],
            radius=20,
            color='red',
            fill=True,
            fill_color='red',
            fill_opacity=0.6
        ).add_child(folium.Popup(record['Launch Site']))
    )
    site_map.add_child(
        folium.map.Marker(
            [record.Lat, record.Long],
            # Create an icon as a text label
            icon=DivIcon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                html=f'<div style="font-size: 12; color:{'red'};"><b>%s</b></div>' % record['Launch Site'],
                )
        )
    )
site_map

## TASK 2: Mark the success/failed launches for each site on the map

In [5]:
spacex_df.tail(2)

,Launch Site,Lat,Long,class
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [6]:
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x==1 else 'red')

In [7]:
spacex_df.tail(2)

,Launch Site,Lat,Long,class,marker_color
54,CCAFS SLC-40,28.563197,-80.57682,1,green
55,CCAFS SLC-40,28.563197,-80.57682,0,red


In [12]:
# Add marker_cluster to current site_map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)
for index, record in spacex_df.iterrows():
    # marker = folium.Marker(...)
    # marker_cluster.add_child(marker)
    marker_cluster.add_child(
        folium.map.Marker(
            [record.Lat, record.Long],
            # Create an icon as a text label
            icon=folium.Icon(
                icon_size=(20,20),
                icon_anchor=(0,0),
                color='white',
                icon_color=record.marker_color
                )
        ).add_child(folium.Popup(record['Launch Site']))
    )    
site_map

## TASK 3: Calculate the distances between a launch site to its proximities

In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)
site_map.add_child(mouse_position)
site_map

In [10]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [11]:
# Location Markers with Distances
launch_coordinates = [28.563197, -80.57682]
site_map.location = launch_coordinates

locations = [{'Name': 'Coast', 'Latitude': 28.563197, 'Longitude': -80.56785, 'Color': 'orange'},
             {'Name': 'City',  'Latitude': 28.0718,   'Longitude': -80.62878, 'Color': 'yellow'},
             {'Name': 'Rail',  'Latitude': 28.57107,  'Longitude': -80.58536, 'Color': 'green' },
             {'Name': 'Road',  'Latitude': 28.56232,  'Longitude': -80.57069, 'Color': 'purple'}]

for location in locations:
    distance = calculate_distance(launch_coordinates[0], launch_coordinates[1], location['Latitude'], location['Longitude'])
    coordinates = [location['Latitude'], location['Longitude']]
    marker = folium.map.Marker(
        coordinates,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12; color:{location['Color']};"><b>%s</b></div>' % f'{distance:.3f} km'
            )
        )
    site_map.add_child(marker)
    lines=folium.PolyLine(locations=[launch_coordinates, coordinates], color=location['Color'], weight=1)
    site_map.add_child(lines)

site_map